<a href="https://colab.research.google.com/github/Weikang01/Do_something_with_tensorflow_and_keras/blob/master/How_Monte_Carlo_Works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##<b>用Monte Carlo随机采样的方式来近似$\pi$</b><br>
在x,y轴各自（-1，1）的正方形区间均匀概率分布中随机采样获得n个点，那么该点落在正方形区间中内接圆的几率是多少呢？<br>
* 通过几何概率分布，我们可以算出：<br>
$$P(x^2+y^2\leqslant1|-1\leqslant x\leqslant1,-1\leqslant y\leqslant1)=\frac{A_{circle}}{A_{square}}$$
$$A_{square}=2^2=2\times2$$
$$A_{circle}=\pi r=\pi1^2=\pi$$
$$\frac{A_{circle}}{A_{square}}=\frac{\pi}{4}$$
* 于是，当采样数为n时，在圆内的点的数量期望为：<br>
$$P_n=\frac{\pi n}{4}$$
* 因此，假设进行n次采样，找到m个在圆内的点，根据大数定律，当n很大时：<br>
$$m\approx \frac{\pi n}{4}$$
$$thus, \frac{4m}{n}\approx\pi$$

In [33]:
import numpy as np
import pandas as pd
n=100000
points=pd.DataFrame(np.random.uniform(-1,1,(n,2)),columns=['x','y'],dtype='float64')
points.head(3)

,x,y
0,-0.421128,-0.827360
1,0.731140,0.345527
2,0.939470,0.621425


In [34]:
points['in_circle']=(points.x**2+points.y**2)<=1
np.mean(points['in_circle']) * 4

3.14408

Monte Carlo随机抽样一个在实际工作中更实际的功能是用来近似求复杂函数的积分<br>
#<b>用Monte Carlo近似求积分</b><br>
#### 先来看看x为一个标量的情况（一元函数）
* 我们有一个函数，e.g.:$$f(x)=\frac{1}{1+\sin(x)\cdot(log_ex)^2}$$
* 计算其定积分：$$I=\int^3_{0.8}f(x)dx$$
* 如果$f(x)$非常复杂，可能无法求出其积分的解析表达式
* 此时，一个好办法是使用Monte Carlo来近似积分结果
<br><br>
> 目标：给定函数$f(x)$，求出其在[a,b]区间的定积分
1. 在[a,b]区间内均匀采样n个样本；记为：$x_1,x_2...$
2. 计算$f(x)$在每个样本$x_i$的函数值，然后对其求平均，再乘以区间大小(b-a)$$Q_n=(b-a)\cdot\frac{1}{n}\sum^n_{i=1}f(x_i)$$
3. 把$Q_n$当作积分结果$I$的近似$$I=\int^b_af(x)dx$$
> Law of large numbers可以保证Monte Carlo的正确性 $Q_n\rightarrow I$，as $ n\rightarrow \infty$<br><br>
###<b> 实际求一下$f(x)=\frac{1}{1+\sin(x)\cdot(log_ex)^2}$在区间[0.8,3]的定积分<b>


In [0]:
def func(x):
  return 1/(1+np.sin(x)*np.log(x)**2)

In [38]:
n=100000
x=np.random.uniform(0.8,3,n).astype('float64')
y=func(x)
y

array([0.74239757, 0.66165115, 0.99244374, ..., 0.67499347, 0.98259467,
       0.74419807])

In [39]:
Q_n=(3-0.8)*np.mean(y)
Q_n

1.7563692128376223

# <b>再来看看$x$为一个向量的情况</b><br>#
> 目标：给定一个多元函数$f(x),(x\in\mathbb{R}^d)$，计算其在集合$\Omega,(\Omega\subset\mathbb{R}^d)$的定积分$I=\int_{\Omega}f(x)dx$

* 方法：<br>
 1. 从集合$\Omega$中均匀抽取n个样本，记为$x_1,...,x_n$
 2. 计算集合$\Omega$的体积：$V=\int_{\Omega}dx.$（注意，这个体积也是一个定积分，这可能和原问题同样困难。如果$\Omega$形状很复杂，我们就无法计算出体积$V$。为了避免这种情况，必须让集合$\Omega$是一个简单的形状，比如长方体、球体等等，这样我们就无需计算定积分，直接用公式就能求出其体积。
 3. 接着求所有n个函数值的平均并乘以体积$V$：$$Q_n=V\cdot\frac{1}{n}\sum^n_{i=1}f(x_i)$$
 4. 返回$Q_n$并将其作为定积分$I$的近似

# <b>使用Monte Carlo来近似期望</b>
#### 期望的定义：

* 设$X$为d维的随机变量，通常把观测值记为小写$x$，而随机变量记为大写$X$
* 函数$p(x)$是随机变量$X$在某个取值点附近的概率密度函数（probability density function, PDF）
 * 其满足$\int_{\mathbb{R}^d}p(x)dx=1$（概率密度函数的性质）
 * 如一元均匀分布的概率密度函数:$p(x)=\frac{1}{t}$,for $x\in[0,t]$.
 * 如一元高斯分布的概率密度函数:$p(x)=\frac{1}{\sigma\sqrt{2\pi}}exp[-\frac{(x-\mu)^2}{2\sigma^2}]$
<br><br>

## <b>近似求期望的步骤</b>
> 目标：估计期望：$\mathbb{E}_{X\sim p}[f(X)]=\int_{\mathbb{R}^b}f(x)\cdot p(x)dx$

1. 根据概率密度函数$p(x)$抽取$n$个随机样本，记为$x_1,...,x_n$
2. 计算$Q_n=\frac{1}{n}\sum^n_{i=1}f(x_i)$.
3. 返回$Q_n$作为期望$\mathbb{E}_{X\sim p}[f(X)]$的近似

## 对Monte Carlo Algorithm的总结
* Monte Carlo的计算结果是错误的
 * 但是它们都接近真实结果
 * 比如随机梯度算法（SGD）就是一种Monte Carlo算法